# ✂️ Tokenization: Hoe AI tekst leest

Taalmodellen zoals ChatGPT, Claude en Gemini lezen geen woorden — ze lezen **tokens**.

Een token is een stukje tekst: soms een heel woord, soms een deel van een woord, soms een leesteken.

In dit notebook ontdek je:
1. Hoe tekst wordt opgesplitst in tokens
2. Waarom sommige woorden in stukjes worden geknipt
3. Hoe padding en truncation werken

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

⚠️ De eerste cel kan even duren (~30 seconden) omdat er een tokenizer wordt gedownload.

In [ ]:
# Installeer de benodigde library (duurt even bij eerste keer)
!pip install transformers -q

## Stap 1: Tekst tokenizen

We laden de **BERT tokenizer** — dezelfde tokenizer die gebruikt wordt in veel AI-modellen.

In [ ]:
from transformers import BertTokenizer

# Laad de BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Probeer een zin
tekst = "PyTorch and Hugging Face make deep learning simple."

# Tokenize!
tokens = tokenizer.tokenize(tekst)

print(f"Originele tekst:  {tekst}")
print(f"Aantal woorden:   {len(tekst.split())}")
print(f"\nTokens:           {tokens}")
print(f"Aantal tokens:    {len(tokens)}")

### Wat valt je op?

- `"PyTorch"` is opgesplitst in `['p', '##yt', '##or', '##ch']` — het model kent het woord niet als geheel
- `"##"` betekent: dit stukje hoort bij het vorige token
- Bekende woorden zoals `"and"`, `"make"`, `"deep"` blijven heel
- De punt `.` is een apart token

## Stap 2: Van tokens naar getallen

Het neural network werkt niet met tekst maar met **getallen**. Elk token heeft een uniek ID-nummer.

In [ ]:
# Zet tokens om naar ID-nummers
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Token → ID-nummer:")
print("-" * 35)
for token, id in zip(tokens, token_ids):
    print(f"  {token:15s} → {id}")

print(f"\nBERT heeft een vocabulaire van {tokenizer.vocab_size:,} tokens!")

## Stap 3: Speciale tokens

BERT voegt automatisch speciale tokens toe:
- `[CLS]` — markering van het begin van de tekst
- `[SEP]` — markering van het einde van de tekst
- `[PAD]` — opvulling om alle zinnen even lang te maken

In [ ]:
# Volledige encoding met speciale tokens en padding
encoded = tokenizer(
    tekst,
    padding='max_length',  # Vul aan tot max_length
    truncation=True,       # Knip af als het te lang is
    max_length=20          # Maximale lengte
)

# Laat zien wat eruit komt
alle_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])

print("Volledige token-reeks (met padding tot lengte 20):")
print("-" * 50)
for i, (token, id, mask) in enumerate(zip(
    alle_tokens, encoded['input_ids'], encoded['attention_mask']
)):
    status = '✅' if mask == 1 else '⬜ (padding)'
    print(f"  [{i:2d}] {token:15s}  ID: {id:5d}  {status}")

print(f"\nAttention mask: {encoded['attention_mask']}")
print("(1 = echte tekst, 0 = opvulling die het model negeert)")

## Stap 4: Terug van getallen naar tekst

We kunnen de getallen ook weer terugvertalen naar tekst.

In [ ]:
# Decodeer terug naar leesbare tekst
decoded = tokenizer.decode(encoded['input_ids'], skip_special_tokens=True)

print(f"Origineel:  {tekst}")
print(f"Gedecodeerd: {decoded}")
print(f"\n(Merk op: hoofdletters zijn verdwenen — BERT werkt met 'uncased' tekst)")

## 🧪 Experimenteer zelf!

Probeer verschillende zinnen te tokenizen. Wat gebeurt er met:
- Nederlandse woorden?
- Spelfouten?
- Getallen?
- Emoji's?

In [ ]:
# Probeer je eigen tekst!
mijn_tekst = "Dit is een Nederlandse zin over kunstmatige intelligentie."  # ← Pas dit aan

mijn_tokens = tokenizer.tokenize(mijn_tekst)
print(f"Tekst:  {mijn_tekst}")
print(f"Tokens: {mijn_tokens}")
print(f"Aantal tokens: {len(mijn_tokens)}")

In [ ]:
# Vergelijk: correct gespeld vs. spelfout
woorden = ["intelligence", "inteligence", "inttellligennce", "Amsterdam", "Amstrdm"]

print("Hoe gaat de tokenizer om met spelfouten?")
print("-" * 50)
for woord in woorden:
    toks = tokenizer.tokenize(woord)
    print(f"  {woord:20s} → {toks}")